In [23]:
import requests
from iso3166 import countries
import iso639

#=== LISTS AND DICTIONARIES OF COUNTRY AND LANGUAGE CODES ===
country_dict = {country.name.title(): country.alpha2.lower() for country in countries}
country_list = sorted([country.name.title() for country in countries])
language_dict = {language.name.title(): language.part1 for language in iso639.ALL_LANGUAGES if language.part1 is not None}
language_list = sorted([language.name.title() for language in iso639.ALL_LANGUAGES if language.part1 is not None])

#=== API PARAMETERS ===
API_KEY = "0223db48f9b941f3a13b6ffa842607aa"
BASE_URL = "https://api.worldnewsapi.com/search-news"

#=== API CALLS ===
def get_search_news_results(query: str, language: str | None = None, country: str | None = None, offset: str = "0"):
    params = {
        "api-key" : API_KEY,
        "text" : f"\"{query.strip()}\"",
        "text-match-indexes" : "title,content",
        "number": "5",
        "offset": offset
    }

    if language is not None:
        params["language"] = language.lower()

    if country is not None:
        params["source-country"] = country.lower()


    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

#=== SEARCH FUNCTION
def search_news():
    choice = input("Q=Quick Search, S=Specific Search: ")
    if choice.lower() == "q":
        query = input("Type in your query: ")
        data = get_search_news_results(query)
        print_results(data)
    elif choice.lower() == "s":
        query = input("Type in your query: ")

        country_code = None
        lang_code = None

        isCountry = False
        while not isCountry:
            country = input("Country: ")
            if country.title() in country_list:
                country_code = country_dict[country.title()]
                isCountry = True
            else:
                print(f"\"{country}\" is not in our database.")
                print("Please recheck the spelling or try its other names.")
        
        isLanguage = False
        while not isLanguage:
            language = input("Language: ")
            if language.title() in language_list:
                lang_code = language_dict[language.title()]
                isLanguage = True
            else:
                print(f"\"{language}\" is not in our database.")
                print("Please recheck the spelling or try its other names.")
        
        data = get_search_news_results(query, lang_code, country_code)
        print_results(data)

def print_results(data):
    print("===== Search Results =====")
    print(f"There are {data["available"]} articles with your query.")
    for entry in data["news"]:
        print(f"\nTitle: {entry["title"]}")
        print(f"Author: {entry["author"]}")
        print(f"Date Published: {entry["publish_date"]}")
        print(f"Language: {next((language for language, code in language_dict.items() if code == entry["language"].lower()), None)} ||", end=" ")
        print(f"Country: {next((country for country, code in country_dict.items() if code == entry["source_country"].lower()), None)}")
        print(f"Text = {entry["text"][:297].replace('\n', ' ') + "..."}")
        print(f"Continue reading at: {entry["url"]}")

def main():
    print("===== News Article Search =====")
    print("\n")
    search_news()

if __name__ == "__main__":
    main()


===== News Article Search =====


"absldbjkalsjd" is not in our database.
Please recheck the spelling or try its other names.
===== Search Results =====
There are 42063 articles with your query.

Title: Dem Lawmakers Get Bomb Threats After Trump Calls For Their Execution
Author: Rachel Kahn
Date Published: 2025-11-22 21:10:28
Language: English || Country: United States Of America
Text = Loomer and Greene had been feuding for months, partially stemming from Loomer’s unabashed support for Israel and Greene’s push to stop arming the country as it carries out a genocide.   Loomer wasn’t coy about her glee at her political opponent’s resignation, posting, “Her life being made miserab...
Continue reading at: https://newrepublic.com/post/203557/dem-lawmakers-bomb-threats-trump-calls-execution

Title: Can a wildlife paradise on a Colombian island survive the arrival of a military base?
Author: Colombia,photographs by Antonio Cascio,Words by Natalia Torres Garzón in Gorgona Island
Date Publishe